# Data Cleaning

Section goals:
- continue cleaning data
- [abandoned?] extract cpu data
- store as cleaned csv files

API and data dictionary:
https://steamspy.com/api.php

Future ideas:
- pc requirements analysis over time
- picture analysis
- keyword/recommender analysis
- categories could make table in a database all on its own, perhaps in future

In [2]:
# load extensions and magics

# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

# http://raw.github.com/jrjohansson/version_information/master/version_information.py
%load_ext version_information
%version_information numpy, pandas

Software versions
Python 3.7.3 64bit [MSC v.1915 64 bit (AMD64)]
IPython 7.5.0
OS Windows 10 10.0.17763 SP0
numpy 1.16.3
pandas 0.24.2
Sat May 04 10:24:26 2019 GMT Summer Time

In [3]:
from ast import literal_eval

import pandas as pd

In [4]:
# customisations
pd.set_option("max_columns", 100)
# pd.reset_option("max_columns")
# pd.options.display.max_columns = 100

In [5]:
raw_steam_data = pd.read_csv('../data/steam_app_data.csv')

print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])
raw_steam_data.head()

Rows: 29235
Columns: 39


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0.0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 65735},{'total': 0},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0.0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 2802},{'total': 0},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/20/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0.0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://steamcdn-a.akamaihd.net/steam/apps/30/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 399, 'final': 3...",[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://steamcdn...",NaN,{'total': 1992},{'total': 0},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://steamcdn-a.akamaihd.net/steam/apps/30/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0.0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</s

In [6]:
def process_null_cols(df, thresh=0.5):
    cutoff_count = len(df) * thresh

    null_counts = df.isnull().sum()
    null_columns = df.columns[null_counts > cutoff_count]
    # print('Dropped columns:', list(null_columns))

    return df.dropna(thresh=cutoff_count, axis=1)


def process_null_row(df, col):
    null_rows = df[df[col].isnull()].shape[0]
    # print("Dropped null '{}' rows:".format(col), null_rows)
    
    return df[df[col].notnull()]
    
    
def process_name(df):
    df = process_null_row(df, 'name')
    # print("Dropped 'none' name rows:", df[df['name'] == 'none'].shape[0])
    return df[df['name'] != 'none']


def process_age(df):
    # UK PEGI Age raings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]
    
    df['required_age'] = pd.cut(df['required_age'], cut_points, labels=label_values)
    return df


def process_release_date(df):
    df = df.copy()
    df['coming_soon'] = df['release_date'].apply(lambda x: literal_eval(x)['coming_soon'])
    # Only want released games
    df = df[df['coming_soon'] == False].copy()
    
    # probably more efficient way to handle this, but works for now
    df['date'] = df['release_date'].apply(lambda x: literal_eval(x)['date'])
    df.loc[df['date'] == '', 'date'] = None
    df['datetime'] = pd.to_datetime(df['date'], format='%d %b, %Y', errors='ignore')
    df['datetime'] = pd.to_datetime(df['date'], format='%b %Y', errors='ignore')
    df['release_date'] = pd.to_datetime(df['datetime'])
    
    df = df.drop(['coming_soon', 'date', 'datetime'], axis=1)
    return df


def convert_price(x):
    try:
        return literal_eval(x)
    except ValueError:
        return {'currency': 'GBP', 'initial': -1}

    
def process_price(df):
    df = df.copy()
    
    # evaluate as dictionary and set to -1 if missing
    df['price_overview'] = df['price_overview'].apply(convert_price)
    
    df['currency'] = df['price_overview'].apply(lambda x: x['currency'])
    df['price'] = df['price_overview'].apply(lambda x: x['initial'])
    
    # Set free games to 0
    df.loc[df['is_free'], 'price'] = 0
    # 4 apps in USD or EUR. Set to -1 for now
    df.loc[df['currency'] != 'GBP', 'price'] = -1
    
    df = df.drop(['is_free', 'currency', 'price_overview'], axis=1)
    
    # store price in pounds, not pennies
    df['price'] /= 100
    
    return df


def is_english(x):
    if 'english' in x.lower():
        return True
    else:
        return False

    
def process_language(df):
    df = df.copy()
    
    # 13 rows with missing data, safe to drop
    df = df.dropna(subset=['supported_languages'])
    
    df['english'] = df['supported_languages'].apply(is_english)
    df = df.drop('supported_languages', axis=1)
    
    return df


steam_data = process_null_cols(raw_steam_data)
steam_data = process_null_row(steam_data, 'type')
steam_data = process_name(steam_data)
steam_data = steam_data.drop('type', axis=1)
steam_data = steam_data.drop_duplicates()
steam_data = process_age(steam_data)
steam_data = process_release_date(steam_data)
steam_data = process_price(steam_data)

# could be useful for future recommender or key-word analysis, but drop for now
steam_data = steam_data.drop(['detailed_description', 'about_the_game', 'short_description'], axis=1)

steam_data = process_language(steam_data)

# links to images. could be useful for future image analysis project
steam_data = steam_data.drop(['header_image', 'screenshots', 'background'], axis=1)

# nearly 10000 missing values, unlikely useful to this analysis
steam_data = steam_data.drop('website', axis=1)

carried_over_steam_data = steam_data.copy()
steam_data.head()

,name,steam_appid,required_age,pc_requirements,mac_requirements,linux_requirements,developers,publishers,packages,package_groups,platforms,categories,genres,movies,achievements,release_date,support_info,content_descriptors,price,english
0,Counter-Strike,10,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...","{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19,True
1,Team Fortress Classic,20,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}","{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99,True
2,Day of Defeat,30,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[30],"[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True
3,Deathmatch Classic,40,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True
4,Half-Life: Opposing Force,50,0,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],[32],"[{'name': 'default', 'title': 'Buy Half-Life: ...","{'windows': True, 'mac': True, 'linux': True}","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...","{'ids': [], 'notes': None}",3.99,True


# Previous work above, new work below

In [7]:
steam_data['pc_requirements']

cols = ['name', 'pc_requirements', 'mac_requirements', 'linux_requirements']

for i in [0, 10, 100, 1000, 10000]:
    for col, val in zip(cols, steam_data.iloc[i][cols]):
        print('-'*20 + col + '-'*(30-len(col)) + '\n')
        try:
            display(literal_eval(val), '\n')
        except:
            display(val, '\n')

--------------------name--------------------------



'Counter-Strike'

'\n'

--------------------pc_requirements---------------



{'minimum': '\r\n\t\t\t<p><strong>Minimum:</strong> 500 mhz processor, 96mb ram, 16mb video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t<p><strong>Recommended:</strong> 800 mhz processor, 128mb ram, 32mb+ video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t'}

...

[]

'\n'

--------------------linux_requirements------------



[]

'\n'

In [8]:
cols = ['pc_requirements']

for i in [0, 10, 100, 1000, 10000]:
    for col, val in zip(cols, steam_data.iloc[i][cols]):
        try:
            display(literal_eval(val))
        except:
            pass

{'minimum': '\r\n\t\t\t<p><strong>Minimum:</strong> 500 mhz processor, 96mb ram, 16mb video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t<p><strong>Recommended:</strong> 800 mhz processor, 128mb ram, 32mb+ video card, Windows XP, Mouse, Keyboard, Internet Connection<br /></p>\r\n\t\t\t'}

{'minimum': '\r\n\t\t\t<p><strong>Minimum: </strong>1.7 GHz Processor, 512MB RAM, DirectX&reg; 8.1 level Graphics Card (Requires support for SSE), Windows&reg; 7 (32/64-bit)/Vista/XP, Mouse, Keyboard, Internet Connection</p>\r\n\t\t\t<p><strong>Recommended: </strong>Pentium 4 processor (3.0GHz, or better), 1GB RAM, DirectX&reg; 9 level Graphics Card, Windows&reg; 7 (32/64-bit)/Vista/XP, Mouse, Keyboard, Internet Connection</p>\r\n\t\t\t'}

{'minimum': '<p><strong>Minimum Requirements:</strong> Windows 98/ME/2000/XP, 128 MB RAM, 500MHz or faster, DirectX: 7.0</p>'}

{'minimum': '<ul class="bb_ul"><li><strong>OS:</strong> Windows XP/Vista/7<br>\t\t\t\t\t\t\t\t\t</li><li><strong>Processor:</strong> 2.0 GHz Core Duo or equivalent processor<br>\t\t\t\t\t\t\t\t\t</li><li><strong>Memory:</strong> 2GB RAM  <br>\t\t\t\t\t\t\t\t\t</li><li><strong>Hard Disk Space:</strong> 4 GB hard disk space<br>\t\t\t\t\t\t\t\t\t</li><li><strong>Video Card:</strong> GeForce 8600 or equivalent Video Card (256 MB of dedicated memory with support for pixel shader 3.0)<br>\t\t\t\t\t\t\t\t\t</li><li><strong>DirectX®:</strong> 9.0<br>\t\t\t\t\t\t\t\t\t</li><li><strong>Sound:</strong> DirectX 9.0 Compatible Sound Card<br>\t\t\t\t\t\t\t\t\t</li><li><strong>Additional:</strong> 3-button mouse, keyboard and speakers<br>\t\t\t\t\t\t\t\t</li></ul>'}

{'minimum': '<strong>Minimum:</strong><br><ul class="bb_ul"><li><strong>OS:</strong> Windows 7 or Greater<br></li><li><strong>Processor:</strong> Intel Core 2 1.06Ghz<br></li><li><strong>Memory:</strong> 1 GB RAM<br></li><li><strong>Graphics:</strong> Integrated graphics<br></li><li><strong>DirectX:</strong> Version 9.0<br></li><li><strong>Storage:</strong> 100 MB available space</li></ul>'}

In [9]:
import re

pattern = r"(?P<cpu>[\.\d]+[ MmGg]+[Hh][Zz])"

for row in steam_data['pc_requirements'][:100]:
    #print(repr(row))
    m = re.findall(pattern, row)
    
    print(m)

['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['500 mhz', '800 mhz']
['1.7 Ghz']
['1.7 GHz', '3.0GHz']
['1.2 GHz', '2.4 GHz']
['1.7 GHz', '3.0GHz']
['1.2 GHz', '2.4 GHz']
['2.4GHz']
['1.7 GHz']
['1.7 GHz', '3.0GHz']
['1.7 GHz', '3.0GHz']
['1.7 GHz', '3.0GHz']
['1.7 GHz', '3.0GHz']
['3.0GHz', '2.4GHz']
['3.0GHz', '2.4GHz']
['2.8 GHz']
['3.0 GHz']
['3.0GHz', '2.4GHz']
[]
['800 MHz', '1500 MHz']
['1.2 GHZ', '2.4 GHZ']
['1.2 GHZ', '2.4 GHZ']
['1.2 GHz', '2.4 GHz']
['600MHz']
[]
[]
['2.0 GHz']
['550Mhz', '1GHz']
[]
['233 Mhz', '300 Mhz']
['233 Mhz', '300 Mhz']
['1Ghz']
['500MHz']
['500 MHz', '900 MHz']
['3.0 GHz']
['2.0 GHz', '2.0 GHz']
['2.0 GHz']
['2.4 GHz', '3.0 GHz']
['233 Mhz', '266 Mhz', '350 Mhz']
['2.0 GHz']
[]
[]
[]
[]
[]
[]
[]
[]
['300 Mhz', '350 Mhz']
[]
[]
[]
['1.8 GHz', '2.8 GHz']
['1.8 GHz', '2.8 GHz']
['3.0GHz', '3.0GHz']
['1.3 GHZ', '2.0 G

In [10]:
import re

def requirements_to_list(x):
    if type(x) == list:
        return None # maybe return empty list?
    else:
        return [i for i in x.items()]
    
def requirements_to_str(x):
    if type(x) == list:
        #print(x)
        return None # maybe return empty list?
    else:
        output = ''
        for i in x.items():
            output += str(i)
        return output
    
def get_cpu(x):
    if x:
        pattern = r"(?P<cpu>[\.\d]+[ MmGg]+[Hh][Zz])"
        m = re.findall(pattern, x)
        print(m)
    return x

def process_requirements(df):
    df = df.copy()
    df['pc_requirements'] = df['pc_requirements'].apply(lambda x: literal_eval(x))
    df['pc_reqs_list'] = df['pc_requirements'].apply(requirements_to_list)
    df['pc_reqs_str'] = df['pc_requirements'].apply(requirements_to_str)
    
    # process cpu
    pattern = r"(?P<cpu>\d?\.?\d+[ MmGg]+[Hh][Zz])"
    df['min_cpu'] = df['pc_reqs_str'].str.extract(pattern)

    df['min_cpu_ghz'] = (df['min_cpu']
                             .str.lower()
                             #.str.replace(' ', '')
                             #.str.replace('mhz', '')
                             #.str.replace('ghz', '00')
                             #.str.replace('.', '')
                             #.astype(float)
                        )

#     df['min_cpu_ghz'] /= 1000
    
    return df



with pd.option_context("display.max_colwidth", 10000):
    # display(temp_df[['name', 'pc_reqs_list']])
    pass

pattern = r"(?P<cpu>[\.\d]+[ MmGg]+[Hh][Zz])"

# temp_df['pc_reqs_str'].str.extract(pattern)
# temp_df['pc_requirements'].isnull().sum()

temp_df = process_requirements(steam_data[:1000])
temp_df.head()

with pd.option_context("display.max_rows", 10000, "display.max_colwidth", 10000):
    # display(temp_df['min_cpu_ghz'].value_counts(dropna=False))
    display(temp_df.loc[temp_df['min_cpu_ghz'].isnull(), 'pc_reqs_str'])

25                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [11]:
pattern = r"Processor:(?:<\/strong>)?[\w\d\s+.]+"
for row in steam_data['pc_requirements'][:100]:
    # print(repr(row))
    m = re.findall(pattern, row)
    print(m)

[]
[]
[]
[]
[]
[]
[]
[]
[]
['Processor:</strong> 1.7 Ghz']
[]
[]
[]
[]
[]
['Processor:</strong> 1.7 GHz Processor']
[]
[]
[]
['Processor:</strong> 1.7 GHz Processor or better', 'Processor:</strong> Pentium 4 processor ']
['Processor:</strong> Pentium 4 3.0GHz                     ', 'Processor:</strong> Intel core 2 duo 2.4GHz                    ']
['Processor:</strong> Pentium 4 3.0GHz', 'Processor:</strong> Intel core 2 duo 2.4GHz']
['Processor:</strong> Dual core from Intel or AMD at 2.8 GHz']
['Processor:</strong> 3.0 GHz P4']
['Processor:</strong> Pentium 4 3.0GHz', 'Processor:</strong> Intel core 2 duo 2.4GHz']
['Processor:</strong> Intel']
[]
['Processor:</strong> 1.2 GHZ or Equivalent', 'Processor:</strong> 2.4 GHZ']
['Processor:</strong> 1.2 GHZ or Equivalent', 'Processor:</strong> 2.4 GHZ or Equivalent']
[]
[]
[]
[]
['Processor:</strong> 2.0 GHz CPU']
[]
[]
[]
[]
[]
[]
[]
['Processor:</strong> 3.0 GHz P4']
[]
['Processor:</strong> Dual Core CPU 2.0 GHz ', 'Processor:</strong> 

In [12]:
def parse_associates(x):
    if x:
        try:
            return literal_eval(x)
        except ValueError:
            return None
    else:
        return None
    
def get_other_associates(x):
    if x:
        if len(x) > 1:
            output = x[1]
            if len(x) > 2:
                for item in x[2:]:
                    output += ', '
                    output += item
            return output
        else:
            return None
    else:
        return None

def process_developers_and_publishers(df):
    df = df[['name', 'developers', 'publishers']].copy() # for test, replace with full df
    
    # handle developers column
    df['developers'] = df['developers'].apply(parse_associates)
    # df['dev_count'] = df['developers'].apply(lambda x: len(x) if x else 0) # for testing, no longer needed
    df['first_developer'] = df['developers'].apply(lambda x: x[0] if x else None)
    df['other_developers'] = df['developers'].apply(get_other_associates)
    
    # handle publishers column
    df['publishers'] = df['publishers'].apply(parse_associates)
    df['first_publisher'] = df['publishers'].apply(lambda x: x[0] if x else None)
    df['other_publishers'] = df['publishers'].apply(get_other_associates)
    
    df = df.drop(['developers', 'publishers'], axis=1)
    
    return df
# num_devs = temp_df['developers'].apply(lambda x: len(list(x)))
# num_devs
temp_df = steam_data.copy()
temp_df = process_developers_and_publishers(temp_df)
# temp_df.head()
temp_df

# temp_df[temp_df['dev_count'] == 1]['developers'].apply(lambda x: x[0])

,name,first_developer,other_developers,first_publisher,other_publishers
0,Counter-Strike,Valve,None,Valve,None
1,Team Fortress Classic,Valve,None,Valve,None
2,Day of Defeat,Valve,None,Valve,None
3,Deathmatch Classic,Valve,None,Valve,None
4,Half-Life: Opposing Force,Gearbox Software,None,Valve,None
5,Ricochet,Valve,None,Valve,None
6,Half-Life,Valve,None,Valve,None
7,Counter-Strike: Condition Zero,Valve,None,Valve,None
8,Half-Life: Blue Shift,Gearbox Software,None,Valve,None
9,Half-Life 2,Valve,None,Valve,None


In [13]:
temp_df = steam_data.copy()
temp_df.head()

temp_df['package_groups'].value_counts()

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [14]:
def process_platforms(df):
    df = df[['name', 'platforms']].copy()
    df['platforms'] = df['platforms'].apply(lambda x: literal_eval(x))
    
    df['windows'] = df['platforms'].apply(lambda x: x['windows'])
    df['mac'] = df['platforms'].apply(lambda x: x['mac'])
    df['linux'] = df['platforms'].apply(lambda x: x['linux'])
    
    df = df.drop('platforms', axis=1)
    
    return df


temp_df = steam_data.copy()

temp_df['platforms'].isnull().sum() # 0
display(temp_df['platforms'].value_counts())
temp_df = process_platforms(temp_df)

temp_df.head()

{'windows': True, 'mac': False, 'linux': False}    19683
{'windows': True, 'mac': True, 'linux': True}       4798
{'windows': True, 'mac': True, 'linux': False}      3644
{'windows': True, 'mac': False, 'linux': True}       633
{'windows': False, 'mac': True, 'linux': False}        5
{'windows': False, 'mac': False, 'linux': True}        1
{'windows': False, 'mac': True, 'linux': True}         1
Name: platforms, dtype: int64

,name,windows,mac,linux
0,Counter-Strike,True,True,True
1,Team Fortress Classic,True,True,True
2,Day of Defeat,True,True,True
3,Deathmatch Classic,True,True,True
4,Half-Life: Opposing Force,True,True,True


In [15]:
def parse_categories(x):
    try:
        return literal_eval(x)
        # cat_list = literal_eval(x)
        
        # display(pd.DataFrame(cat_list))
        # for cat in cat_list:
        #    print(cat['description'])
        # return [c['description'] for c in cat_list]
        # return ', '.join(c['description'] for c in cat_list)
    except ValueError:
        return None

def process_categories(df):
    df = df[['name', 'steam_appid', 'categories']].copy()
    
    df['categories_parsed'] = df['categories'].apply(parse_categories)
    
    categories = {}
    for i, row in df.iterrows():
        appid = row['steam_appid']
        #display(df.iloc[i])
        for cat in row['categories_parsed']:
            col_name = cat['description']
            categories.setdefault(appid, []).append(col_name)
            
    display(categories)
            
    
    return df

temp_df = steam_data.copy()
temp_df.head()

temp_df['categories'].isnull().sum() # 553
temp_df['categories'].value_counts(dropna=False)

temp_df = process_categories(temp_df[:5])
temp_df.head()

# temp_df['categories'].value_counts(dropna=False)
# temp_df[temp_df['categories'].isnull()]
# pd.DataFrame(temp_df.categories_parsed.to_list())

{10: ['Multi-player',
  'Online Multi-Player',
  'Local Multi-Player',
  'Valve Anti-Cheat enabled'],
 20: ['Multi-player',
  'Online Multi-Player',
  'Local Multi-Player',
  'Valve Anti-Cheat enabled'],
 30: ['Multi-player', 'Valve Anti-Cheat enabled'],
 40: ['Multi-player',
  'Online Multi-Player',
  'Local Multi-Player',
  'Valve Anti-Cheat enabled'],
 50: ['Single-player', 'Multi-player', 'Valve Anti-Cheat enabled']}

,name,steam_appid,categories,categories_parsed
0,Counter-Strike,10,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': 1, 'description': 'Multi-player'}, {'i..."
1,Team Fortress Classic,20,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': 1, 'description': 'Multi-player'}, {'i..."
2,Day of Defeat,30,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': 1, 'description': 'Multi-player'}, {'i..."
3,Deathmatch Classic,40,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': 1, 'description': 'Multi-player'}, {'i..."
4,Half-Life: Opposing Force,50,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': 2, 'description': 'Single-player'}, {'..."


In [16]:
temp_df = steam_data.copy()

row_list = []

col_names = {}

for i, row in temp_df.iterrows():
    try:
        list_of_dicts = literal_eval(row['categories'])
    except ValueError:
        continue
        
    new_col_dict = {}

    for cat_dict in list_of_dicts:
        col_name = cat_dict['description']
        col_names[col_name] = []

col_names

cat_cols = pd.DataFrame(columns=col_names)
display(list(cat_cols.columns))

['Multi-player',
 'Online Multi-Player',
 'Local Multi-Player',
 'Valve Anti-Cheat enabled',
 'Single-player',
 'Steam Cloud',
 'Steam Achievements',
 'Steam Trading Cards',
 'Captions available',
 'Partial Controller Support',
 'Includes Source SDK',
 'Cross-Platform Multiplayer',
 'Stats',
 'Commentary available',
 'Includes level editor',
 'Steam Workshop',
 'In-App Purchases',
 'Co-op',
 'Full controller support',
 'Steam Leaderboards',
 'SteamVR Collectibles',
 'Online Co-op',
 'Shared/Split Screen',
 'Local Co-op',
 'MMO',
 'VR Support',
 'Mods',
 'Mods (require HL2)',
 'Steam Turn Notifications']

In [17]:
temp_df = steam_data.copy()

cat_cols = pd.DataFrame()

for i, row in temp_df.iterrows():
    if i % 1000 == 0:
        print('\r', i, end='')
    try:
        list_of_dicts = literal_eval(row['categories'])
    except ValueError:
        continue
        
    new_col_dict = {}
    new_col_dict['steam_appid'] = row['steam_appid']
    for cat_dict in list_of_dicts:
        new_col_dict[cat_dict['description']] = True
    
    cat_cols = pd.concat([cat_cols, pd.Series(new_col_dict)], axis=1, sort=False, ignore_index=True)

cat_cols = cat_cols.fillna(False)    
category_data = cat_cols.T

category_data.to_csv('../data/steam_category_data.csv', index=False)
category_data.head()
    
# d1 = col_dicts[0]
# d2 = col_dicts[1]

# pd.concat([pd.Series(d1), pd.Series(d2)], axis=1).T
# pd.concat([cat_cols, pd.Series(d2)], axis=1)

 29000

,steam_appid,Multi-player,Online Multi-Player,Local Multi-Player,Valve Anti-Cheat enabled,Single-player,Steam Cloud,Steam Achievements,Steam Trading Cards,Captions available,Partial Controller Support,Includes Source SDK,Cross-Platform Multiplayer,Stats,Commentary available,Includes level editor,Steam Workshop,In-App Purchases,Co-op,Full controller support,Steam Leaderboards,SteamVR Collectibles,Online Co-op,Shared/Split Screen,Local Co-op,MMO,VR Support,Mods,Mods (require HL2),Steam Turn Notifications
0,10,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,20,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,30,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,40,True,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,50,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [80]:
def parse_categories(x):
    try:
        return {c['description']:1 for c in literal_eval(x)}
    except ValueError:
        return {}

    
def get_col_list(series):
    cols_dict = {}
    
    def create_col_dict(x):
        for item in x.keys():
            cols_dict[item] = 1
    
    series.apply(create_col_dict)
    
    return list(cols_dict.keys())
    
    
def process_categories(df):
    df = df[['name', 'steam_appid', 'categories']].copy()
    
    df['categories'] = df['categories'].apply(parse_categories)
    col_list = get_col_list(df['categories'])
    print(col_list)
    
    def set_category(x, col):
        if col in x.keys():
            return 1
        else:
            return 0
    
    # this has potential, fairly vast vectorised method. just need to automate for all categories
    # col = 'Single-player'
    # df['single_player'] = df['categories'].apply(set_category, args=(col,))
    # col_list = ['Multi-player', 'Online Multi-Player', 'Local Multi-Player', 'Valve Anti-Cheat enabled', 'Single-player', 'Steam Cloud', 'Steam Achievements', 'Steam Trading Cards', 'Captions available', 'Partial Controller Support', 'Includes Source SDK', 'Cross-Platform Multiplayer', 'Stats', 'Commentary available', 'Includes level editor', 'Steam Workshop', 'In-App Purchases', 'Co-op', 'Full controller support', 'Steam Leaderboards', 'SteamVR Collectibles', 'Online Co-op', 'Shared/Split Screen', 'Local Co-op', 'MMO', 'VR Support', 'Mods', 'Mods (require HL2)', 'Steam Turn Notifications']

    for col in col_list:
        col_name = (col.lower()
                       .replace('-', '_')
                       .replace(' ', '_')
                       .replace('(', '')
                       .replace(')', '')
                       .replace('/', '_or_')
                   )
        df[col_name] = df['categories'].apply(set_category, args=(col,))
    
    # transform could be useful
    # new_df = df['categories'].transform([lambda x: {c['description']:1 for c in x}, lambda x: 0])
    # df = pd.concat([df, new_df], axis=1)
    
    df = df.drop('categories', axis=1)
    
    return df

temp_df = process_categories(steam_data) # 385:390 has missing for testing
temp_df.head()

['Multi-player', 'Online Multi-Player', 'Local Multi-Player', 'Valve Anti-Cheat enabled', 'Single-player', 'Steam Cloud', 'Steam Achievements', 'Steam Trading Cards', 'Captions available', 'Partial Controller Support', 'Includes Source SDK', 'Cross-Platform Multiplayer', 'Stats', 'Commentary available', 'Includes level editor', 'Steam Workshop', 'In-App Purchases', 'Co-op', 'Full controller support', 'Steam Leaderboards', 'SteamVR Collectibles', 'Online Co-op', 'Shared/Split Screen', 'Local Co-op', 'MMO', 'VR Support', 'Mods', 'Mods (require HL2)', 'Steam Turn Notifications']


,name,steam_appid,multi_player,online_multi_player,local_multi_player,valve_anti_cheat_enabled,single_player,steam_cloud,steam_achievements,steam_trading_cards,captions_available,partial_controller_support,includes_source_sdk,cross_platform_multiplayer,stats,commentary_available,includes_level_editor,steam_workshop,in_app_purchases,co_op,full_controller_support,steam_leaderboards,steamvr_collectibles,online_co_op,shared_or_split_screen,local_co_op,mmo,vr_support,mods,mods_require_hl2,steam_turn_notifications
0,Counter-Strike,10,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Team Fortress Classic,20,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Day of Defeat,30,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Deathmatch Classic,40,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Half-Life: Opposing Force,50,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
col_list = ['Single-player', 'Multi-player']

for col in col_list:
    col = col.lower()
    col = col.replace('-', '_')
    # print(col)
    
col_list = ['Multi-player',
 'Online Multi-Player',
 'Local Multi-Player',
 'Valve Anti-Cheat enabled',
 'Single-player',
 'Steam Cloud',
 'Steam Achievements',
 'Steam Trading Cards',
 'Captions available',
 'Partial Controller Support',
 'Includes Source SDK',
 'Cross-Platform Multiplayer',
 'Stats',
 'Commentary available',
 'Includes level editor',
 'Steam Workshop',
 'In-App Purchases',
 'Co-op',
 'Full controller support',
 'Steam Leaderboards',
 'SteamVR Collectibles',
 'Online Co-op',
 'Shared/Split Screen',
 'Local Co-op',
 'MMO',
 'VR Support',
 'Mods',
 'Mods (require HL2)',
 'Steam Turn Notifications']

for col in col_list:
    col = col.lower()
    col = col.replace('-', '_').replace(' ', '_').replace('(', '').replace(')', '').replace('/', '_or_')
    # print(col)
    
def get_col_list(df):
    categories = df['categories'].copy()
    categories = categories.apply(literal_eval)
    
    cols_dict = {}
    
    def col_func(x):
        cat_list = [item['description'] for item in x]
        for item in cat_list:
            cols_dict[item] = 1
        
        return None
    
    categories.apply(col_func)
    
    return list(cols_dict.keys())

temp_df = steam_data[['name', 'steam_appid', 'categories']].copy()
temp_df.head()

col_list = get_col_list(temp_df[:300])
col_list

['Multi-player',
 'Online Multi-Player',
 'Local Multi-Player',
 'Valve Anti-Cheat enabled',
 'Single-player',
 'Steam Cloud',
 'Steam Achievements',
 'Steam Trading Cards',
 'Captions available',
 'Partial Controller Support',
 'Includes Source SDK',
 'Cross-Platform Multiplayer',
 'Stats',
 'Commentary available',
 'Includes level editor',
 'Steam Workshop',
 'In-App Purchases',
 'Co-op',
 'Full controller support',
 'Steam Leaderboards',
 'SteamVR Collectibles',
 'Online Co-op',
 'Shared/Split Screen',
 'Local Co-op',
 'MMO']

# Final processing functions
(for this section)

In [87]:
def process_requirements(df):
    df = df.copy()
    # drop requirements for now, can return to later
    df = df.drop(['pc_requirements', 'mac_requirements', 'linux_requirements'], axis=1)
    return df


def parse_associates(x):
    if x:
        try:
            return literal_eval(x)
        except ValueError:
            return None
    else:
        return None


def get_other_associates(x):
    if x:
        if len(x) > 1:
            output = x[1]
            if len(x) > 2:
                for item in x[2:]:
                    output += ', '
                    output += item
            return output
        else:
            return None
    else:
        return None

    
def process_developers_and_publishers(df):
    df = df.copy()
    
    # handle developers column
    df['developers'] = df['developers'].apply(parse_associates)
    df['developer'] = df['developers'].apply(lambda x: x[0] if x else None)
    df['other_developers'] = df['developers'].apply(get_other_associates)
    
    # handle publishers column
    df['publishers'] = df['publishers'].apply(parse_associates)
    df['publisher'] = df['publishers'].apply(lambda x: x[0] if x else None)
    df['other_publishers'] = df['publishers'].apply(get_other_associates)
    
    df = df.drop(['developers', 'publishers'], axis=1)
    
    return df


def process_packages(df):
    df = df.copy()
    # not useful for now, drop package columns
    df = df.drop(['packages', 'package_groups'], axis=1)
    return df


def process_platforms(df):
    df = df.copy()
    df['platforms'] = df['platforms'].apply(lambda x: literal_eval(x))
    
    df['windows'] = df['platforms'].apply(lambda x: x['windows'])
    df['mac'] = df['platforms'].apply(lambda x: x['mac'])
    df['linux'] = df['platforms'].apply(lambda x: x['linux'])
    
    df = df.drop('platforms', axis=1)
    
    return df


def parse_categories(x):
    try:
        return {c['description']:1 for c in literal_eval(x)}
    except ValueError:
        return {}

    
def get_col_list(series):
    cols_dict = {}
    
    def create_col_dict(x):
        for item in x.keys():
            cols_dict[item] = 1
    
    series.apply(create_col_dict)
    
    return list(cols_dict.keys())
    
    
def process_categories(df):
    df = df[['steam_appid', 'categories']].copy()
    
    df['categories'] = df['categories'].apply(parse_categories)
    col_list = get_col_list(df['categories'])
    
    def set_category(x, col):
        if col in x.keys():
            return 1
        else:
            return 0
    
    for col in col_list:
        col_name = (col.lower()
                       .replace('-', '_')
                       .replace(' ', '_')
                       .replace('(', '')
                       .replace(')', '')
                       .replace('/', '_or_')
                   )
        df[col_name] = df['categories'].apply(set_category, args=(col,))
    
    df = df.drop('categories', axis=1)
    
    return df


steam_data = carried_over_steam_data.copy()

steam_data = process_requirements(steam_data)
steam_data = process_developers_and_publishers(steam_data)
steam_data = process_packages(steam_data)
steam_data = process_platforms(steam_data)

category_data = process_categories(steam_data)
category_data.to_csv('../data/steam_category_data.csv', index=False)
steam_data = steam_data.drop('categories', axis=1)

display(category_data.head())
steam_data.head()

,steam_appid,multi_player,online_multi_player,local_multi_player,valve_anti_cheat_enabled,single_player,steam_cloud,steam_achievements,steam_trading_cards,captions_available,partial_controller_support,includes_source_sdk,cross_platform_multiplayer,stats,commentary_available,includes_level_editor,steam_workshop,in_app_purchases,co_op,full_controller_support,steam_leaderboards,steamvr_collectibles,online_co_op,shared_or_split_screen,local_co_op,mmo,vr_support,mods,mods_require_hl2,steam_turn_notifications
0,10,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,20,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,30,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,40,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,50,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


,name,steam_appid,required_age,genres,movies,achievements,release_date,support_info,content_descriptors,price,english,developer,other_developers,publisher,other_publishers,windows,mac,linux
0,Counter-Strike,10,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2000-11-01,"{'url': 'http://steamcommunity.com/app/10', 'e...","{'ids': [2, 5], 'notes': 'Includes intense vio...",7.19,True,Valve,None,Valve,None,True,True,True
1,Team Fortress Classic,20,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-04-01,"{'url': '', 'email': ''}","{'ids': [2, 5], 'notes': 'Includes intense vio...",3.99,True,Valve,None,Valve,None,True,True,True
2,Day of Defeat,30,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2003-05-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True,Valve,None,Valve,None,True,True,True
3,Deathmatch Classic,40,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},2001-06-01,"{'url': '', 'email': ''}","{'ids': [], 'notes': None}",3.99,True,Valve,None,Valve,None,True,True,True
4,Half-Life: Opposing Force,50,0,"[{'id': '1', 'description': 'Action'}]",NaN,{'total': 0},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...","{'ids': [], 'notes': None}",3.99,True,Gearbox Software,None,Valve,None,True,True,True
